In [261]:
import pandas as pd
import numpy as np
import json

In [262]:
pd.set_option('display.max_columns', 5000)

In [263]:
def comparison(playerid, matchId, file, cols):
    total=pd.read_csv(file)
    m=total[cols].mean()
    print('The mean:')
    print(m)
    total=total[total['playerId']==playerid]
    total=total[total['matchId']==matchId]
    path='data/events/'+str(matchId)+'.json'
    with open(path, 'r', encoding='utf-8') as f:
        rd = f.read()
    rd=rd[28:-2]
    obj = json.loads(rd)
    dfw = pd.json_normalize(obj)
    dfw=dfw[dfw['player.id']==playerid]
    print('player:', dfw['player.name'].iloc[0])
    print(total[cols])
    

In [264]:
def get_player(playerid, matchId):

    path='data/events/'+str(matchId)+'.json'
    with open(path, 'r', encoding='utf-8') as f:
        rd = f.read()
    rd=rd[28:-2]
    obj = json.loads(rd)
    dfw = pd.json_normalize(obj)
    dfw=dfw[dfw['player.id']==playerid]
    return dfw['player.name'].iloc[0]

In [265]:
def bestplayers(df):
    for i in range (15):
        id=df['playerId'].iloc[i]
        matchid=df['matchId'].iloc[i]
        player=get_player(id,matchid)
        print('matchid:',matchid)
        print('playerid:', id)
        print(player)

In [266]:
loss_list= [
    'total.losses',
    'total.ownHalfLosses',
    'total.dangerousOwnHalfLosses'
]

In [295]:
def preprocessdf(mean, std, position):
    meandf=pd.read_csv(mean)
    stddf=pd.read_csv(std)
    pdf=pd.read_csv(position)
    meandf=meandf[meandf.columns[1:]]
    stddf=stddf[stddf.columns[1:]]
    '''
    pdf=pdf[pdf.columns[1:]]
    '''
    keep_columns=meandf.columns
    calc_col = [col for col in pdf.columns if col.startswith('total.') and col != 'total.minutesOnField']
    calc_filtered = [col for col in calc_col if col not in keep_columns]
    cpdf=pdf.drop(columns=calc_filtered)
    for w in loss_list:
        if(w in keep_columns):
            cpdf[w]*=-1
    return cpdf, meandf, stddf

In [296]:
def resultcalculation(calc,mean,std, weight):
    t0=calc.copy()
    t1=calc.copy()
    t2=calc.copy()
    
    t0[mean.columns]-=mean.loc[0]
    t1[mean.columns]-=mean.loc[1]
    t2[mean.columns]-=mean.loc[2]
    t0[mean.columns]/=std.loc[0]
    t1[mean.columns]/=std.loc[1]
    t2[mean.columns]/=std.loc[2]
    '''
    '''
    vec=np.array(weight)
    calc_col = [col for col in calc.columns if col.startswith('total.') and col != 'total.minutesOnField']
    statsarray0=t0[calc_col].to_numpy()
    statsarray1=t1[calc_col].to_numpy()
    statsarray2=t2[calc_col].to_numpy()
    res0=np.dot(statsarray0, vec)
    res1=np.dot(statsarray1, vec)
    res2=np.dot(statsarray2, vec)
    return res0, res1, res2

In [411]:
meanpath='mean_std/mean_midfieldclustering.csv'
stdpath='mean_std/std_midfieldclustering.csv'
playerpath='norm_stats/normalized_midfieldstats.csv'

In [412]:
calcdf, meandf, stddf=preprocessdf(meanpath, stdpath, playerpath)

In [415]:
#todo
lb_run_weights=[]



lb_def_weights=[1,2,1,4,1,4,1,4,1,1,1,2,4,1,4,1,2,4,1,1,4,1,0,0,1,1]
lb_off_weights=[1,4,1,2,1,2,2,4,2,4,1,4,2,1,2,1,2,2,4,2,2,1,4,4,1,1]
iv_def_weights=[1,2,1,4,1,4,1,4,2,1,4,1,2,4,1,2,4,1,0,1,1]
iv_hyb_weights=[1,2,1,4,1,4,1,4,2,1,4,1,4,1,1,2,4,1,0,1,2]
iv_off_weights=[1,4,1,4,1,1,1,4,4,1,4,1,4,1,2,2,4,2,2,1,2]
wi_pla_weights=[1,1,4,1,1,4,2,1,4,2,4,4,2,4,4]
wi_dri_weights=[1,1,4,1,4,2,4,1,4,4,4,2,4,4,2]
wi_run_weights=[1,1,4,1,2,2,4,1,4,4,4,1,4,4,4]
st_run_weights=[1,0,2,2,4,2,1,4,4,4,4,1,4]
st_goa_weights=[1,0,2,2,4,2,4,4,4,2,2,1,4]
st_box_weights=[2,0,2,2,2,1,1,4,2,2,4,1,4]
mi_dpl_weigts=[]
mi_opl_weigts=[]
mi_def_weigts=[]
mi_off_weigts=[]
unitlist=[1]*len(st_run_weights)

In [413]:
featuredf=pd.DataFrame(columns=meandf.columns)
'''
featuredf.loc[0]=iv_def_weights
featuredf.loc[1]=iv_hyb_weights
featuredf.loc[2]=iv_off_weights
'''
featuredf

,total.defensiveDuels,total.defensiveDuelsWon,total.offensiveDuels,total.passes,total.successfulPasses,total.passesToFinalThird,total.forwardPasses,total.successfulForwardPasses,total.throughPasses,total.successfulThroughPasses,total.keyPasses,total.longPasses,total.successfulLongPasses,total.interceptions,total.pressingDuels,total.recoveries,total.opponentHalfRecoveries,total.losses,total.ownHalfLosses,total.xgShot,total.xgAssist,total.receivedPass,total.touchInBox,total.progressiveRun


In [406]:
r0,r1,r2=resultcalculation(calcdf,meandf,stddf, st_goa_weights)

In [407]:
calcdf['Res0def']=r0
calcdf['Res1def']=r1
calcdf['Res2def']=r2

In [408]:
ranking=calcdf.sort_values(by='Res0def', ascending=False)

In [409]:
bestplayers(ranking)

matchid: 2800011
playerid: 101669
A. Dzyuba
matchid: 2799923
playerid: 3291
Paco Alcácer
matchid: 2799578
playerid: 8717
H. Kane
matchid: 2800021
playerid: 454072
A. Georgiou
matchid: 5414162
playerid: 273206
G. Rodrigues
matchid: 2799552
playerid: 8717
H. Kane
matchid: 2799965
playerid: 97592
M. Dabbur
matchid: 2800028
playerid: 101669
A. Dzyuba
matchid: 2799664
playerid: 118
M. Depay
matchid: 2800067
playerid: 14879
T. Pukki
matchid: 2799667
playerid: 234710
C. Washington
matchid: 2799660
playerid: 135923
Gonçalo Paciência
matchid: 2799654
playerid: 3322
Cristiano Ronaldo
matchid: 2799965
playerid: 20824
E. Zahavi
matchid: 5414119
playerid: 442514
L. Dykes


In [ ]:
path="norm_stats/adjustedstrikerstats.csv"
comparison(101669,2800011,path, meandf.columns)

In [410]:
ranking.to_csv('rank_st_zielspielerweights.csv',index=False)